In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('eng_-french.csv')

In [145]:
df.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [146]:
batch_size = 64
epochs = 100
latent_dim = 256

In [147]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [148]:
index = 0
for i in df["English words/sentences"]:
    index+=1
    input_texts.append(i)
    for j in i:
        if j not in input_characters:
            input_characters.add(j)
    if index > len(df)/8:
        print(index)
        break

21953


In [149]:
index = 0
for i in df["French words/sentences"]:
    index+=1
    target_texts.append(i)
    for j in i:
        if j not in target_characters:
            target_characters.add(j)
    if index > len(df)/8:
        print(index)
        break

21953


In [150]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [151]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [152]:
max_encoder_seq_length = max([len(i) for i in input_texts])
max_decoder_seq_length = max([len(i) for i in target_texts])

In [153]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 21953
Number of unique input tokens: 74
Number of unique output tokens: 100
Max sequence length for inputs: 18
Max sequence length for outputs: 57


In [154]:
input_token_index = dict([(i, j) for j, i in enumerate(input_characters)])
target_token_index = dict([(i, j) for j, i in enumerate(target_characters)])

In [155]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [156]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [157]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [158]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=100,
    validation_split=0.2,
)
model.save("m1")

Epoch 1/100
275/275 [==============================] - 132s 466ms/step - loss: 1.0157 - accuracy: 0.7403 - val_loss: 0.9326 - val_accuracy: 0.7338
Epoch 2/100
275/275 [==============================] - 127s 462ms/step - loss: 0.6912 - accuracy: 0.8000 - val_loss: 0.7876 - val_accuracy: 0.7696
Epoch 3/100
275/275 [==============================] - 128s 464ms/step - loss: 0.5927 - accuracy: 0.8257 - val_loss: 0.6957 - val_accuracy: 0.7957
Epoch 4/100
275/275 [==============================] - 127s 463ms/step - loss: 0.5303 - accuracy: 0.8426 - val_loss: 0.6526 - val_accuracy: 0.8067
Epoch 5/100
275/275 [==============================] - 127s 462ms/step - loss: 0.4839 - accuracy: 0.8558 - val_loss: 0.5920 - val_accuracy: 0.8236
Epoch 6/100
275/275 [==============================] - 127s 461ms/step - loss: 0.4466 - accuracy: 0.8663 - val_loss: 0.5606 - val_accuracy: 0.8327
Epoch 7/100
275/275 [==============================] - 127s 464ms/step - loss: 0.4166 - accuracy: 0.8754 - val_loss: 0

INFO:tensorflow:Assets written to: m1\assets


INFO:tensorflow:Assets written to: m1\assets


In [159]:
model = keras.models.load_model("m1")

In [160]:
encoder_inputs = model.input[0] 
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [161]:
reverse_input_char_index = dict((i, j) for j, i in input_token_index.items())
reverse_target_char_index = dict((i, j) for j, i in target_token_index.items())

In [176]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    return decoded_sentence

In [163]:
import random

In [165]:
model2=keras.models.load_model("m1")

In [177]:
for i in range(10):
    seq_index = random.randint(0,21953)
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("Стало:",input_texts[seq_index],"==>",decoded_sentence)
    print("Было:",df["English words/sentences"][seq_index],"=",df["French words/sentences"][seq_index],"\n")

Стало: He ratted us out. ==> lle vons boigolé.                                         
Было: He ratted us out. = Il nous a dénoncés. 

Стало: It's 3:30. ==> 'est un rengerçais.                                       
Было: It's 3:30. = Il est trois heures et demie. 

Стало: Tom's drowning. ==> la vote blassé.                                           
Было: Tom's drowning. = Tom se noie. 

Стало: Hold on a minute. ==> onsez-vous un moment.                                     
Было: Hold on a minute. = Attends un peu ! 

Стало: I remember. ==> e miente d'aucire.                                        
Было: I remember. = Je me le rappelle. 

Стало: I'm intrigued. ==> le suis venu en sortir.                                   
Было: I'm intrigued. = Je suis intrigué. 

Стало: You're so sweet. ==> e semblef se le travaie.                                  
Было: You're so sweet. = Vous êtes tellement gentille ! 

Стало: Thanks for coming. ==> erche mon travail.                              